# Sentinel-2 Analysis with OpenGeo (Element84)

This notebook demonstrates how to perform Sentinel-2 imagery analysis using the `opengeo` package with the **Element84 Earth Search** STAC API.

## 1. Initialize OpenGeo

In [ ]:
import opengeo as og

# Initialize with Element84 Earth Search
og.Initialize("https://earth-search.aws.element84.com/v1")

## 2. Define ROI

Define a region over the Grand Canyon.

In [ ]:
roi = og.Geometry.Rectangle(-112.15, 36.05, -112.05, 36.15)
print("ROI:", roi)

## 3. Load Sentinel-2 Collection

Sentinel-2 L2A collection on Earth Search is `sentinel-2-c1-l2a`.

In [ ]:
s2_col = og.ImageCollection("sentinel-2-c1-l2a") \
    .filterDate("2023-05-01", "2023-05-31") \
    .filterBounds(roi) \
    .select(["red", "green", "blue", "nir"])

print(f"Images found: {s2_col.size()}")

## 4. True Color Visualization

Combine bands for RGB visualization. We specify `epsg=32612` (UTM Zone 12N) to ensure a consistent coordinate system.

In [ ]:
# Specify EPSG to avoid CRS ambiguity
s2_image = s2_col.median(epsg=32612)

Map = og.Map()
Map.addLayer(s2_image, {'bands': ['red', 'green', 'blue'], 'min': 0, 'max': 3000}, "Sentinel-2 RGB")
Map.centerObject(roi, zoom=12)
Map

## 5. Calculate NDVI

In [ ]:
ndvi = s2_image.normalizedDifference(["nir", "red"]).rename("NDVI")
Map.addLayer(ndvi, {'min': 0, 'max': 0.8, 'palette': 'red,yellow,green'}, "Sentinel-2 NDVI")
Map